<h1>PROSES EXTRACT DATA</h1>

<h3>1. Inisialisasi dengan import library yang diperlukan</h3>

In [17]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import time

<h3>2. Pengambilan data page source dari link yang diinginkan</h3>

In [2]:
driver = webdriver.Chrome()
link = 'https://id.wikipedia.org/wiki/Daftar_gempa_bumi_di_Indonesia'
driver.get(link)
time.sleep(3)
content = driver.page_source
driver.quit()

<h3>3. Melakukan parsing terhadap data page source yang diperoleh menggunakan BeautifulSoup supaya dapat diambil setiap tag htmlnya. Lalu diambil tag yang memiliki data table</h3>

In [4]:
soup = BeautifulSoup(content,'html.parser')
tables = soup.find_all('table', {'class':'wikitable sortable jquery-tablesorter'})

<h3>4. Mengambil nama kolom dari table lalu dibuat dataframe</h3>

In [54]:
coloumn = [data.text.strip() for data in tables[0].find('thead').find_all('th')]
df = pd.DataFrame(columns= coloumn)

<h3>5. Proses pengambilan data dari tag html </h3>

<p>Pada table terdapat kolom yang memiliki atribut rowspan, kolom tersebut merupakan kolom yang marge dengan kolom row di bawahnya dengan jumlah sebanyak value dari attribut spannya. Untuk mengatasi hal ini langkah yang dilakukan yaitu menyimpan value kolom yang memiliki atribut rowspan ke dalam list copy dengan index sesuai dengan aslinya.</p>

In [55]:
#Membuat list copy dengan panjang list sesuai dengan panjang dataframe
#dalam menyimpan value yang di copy perlu di samakan juga index dari value datanya supaya posisinya benar ketika dicopy ke bawah
copy = ['']*len(coloumn)
#ini merupakan indikator untuk iterasi row melakukakn copy data atau tidak
rowspan = 0

for body in tables:
  body_table = body.find('tbody')
  raw_data = body_table.find_all('tr')

  #iterasi setiap row
  for row in raw_data:
    data = row.find_all('td')
    row_data = ['']*len(coloumn) #List ini digunakan untuk menampung setiap value dari kolom.
    i_cell = 0
    i = 0

    #pengecekan apakah pada baris ini perlu dilakukan copy data dari list copy
    if not rowspan:
      for i, cell in enumerate(data): #iterasi setiap data kolom
        span = cell.get('rowspan') #pengambilan atribut rowspan

        #pengecekan apakah kolom memiliki
        if span:
          rowspan = int(span)-1 #jika ada maka value dari rowspon akan direwrite ke variable rowspan
          copy[i]=cell.text.strip() #value dari kolom dicopy ke list copy
          row_data[i]=cell.text.strip() #Lalu dimasukkan ke row_data 
        else:
          row_data[i]=cell.text.strip() #jika kolom tidak ada rowspan maka value kolom akan langsung dimasukkan ke row_data
    
    #Ini merupakan kondisi jika baris memerlukan copy value dari kolom baris sebelumnya dengan indikator rowspan tidak 0 
    else:
      copy2 = ['']*len(coloumn) #list ini disiapkan jika pada baris ini ditemukan lagi atribut rowspan dalam kolom
      row_data = ['']*len(coloumn) #List ini digunakan untuk menampung setiap value dari kolom.
      for x, cell in enumerate(data): #iterasi setiap data kolom
        span = cell.get('rowspan') #pengambilan atribut rowspan
        if span:
          copy2[x] = cell.text.strip() #jika ada kolom beratribut rowspan maka akan disimpak ke copy2
      
      #proses penggabungan value dari list copy ke value setiap kolom di baris ini
      list_cell = [c.text.strip()for c in data] #Value kolom data dimasukkan ke dalam list
      index_cell = 0
      #proses penggabungan
      for index in range(len(copy)):
          if copy[index] == '' and index_cell < len(list_cell): #Jika pada list copy pada index saat ini kosong
            row_data[index] = list_cell[index_cell] #value yang ddimasukkan ke row_data yaitu dari value kolom yang asli
            index_cell +=1
          else:
            row_data[index] = copy[index] #jika list copy pada index saat ini isi, maka akan dimasukkan ke dalam  row_data

      rowspan -= 1
      #jika rowspan sudah mencapai target banyak copy maka list copy akan dikembalikan kosong
      if rowspan == 0:
        copy = ['']*len(coloumn)

      #penggabungan copy2 ke copy
      for i in range(len(copy)):
        if copy[i] == '' and copy2[i] != '': 
          copy[i] = copy2[i]
    #proses memasukkan data setiap baris (row_data) ke dalam dataframe sengan method loc
    length = len(df)
    df.loc[length] = row_data


In [56]:
df.head(10)

,Tanggal,Lokasi,Kekuatan,Kedalaman (km),MMI,Korban Jiwa,Korban Luka,Keterangan
0,2024-09-18,Jawa Barat,5.1 Mw,10.0,VI,2,159,Gempa bumi Jawa Barat 2024
1,2024-09-07,Bali,4.8 Mw,35.8,VI,1,3,10 bangunan rusak.[6][7]
2,2024-04-27,Jawa Barat,6.1 Mw,54.3,V,1,12,Ratusan bangunan rusak.[8][9]
3,2024-03-22,Jawa Timur,6.4 Mw[10],9.5,VIII,,4,Gempa bumi Laut Jawa 2024
4,2024-02-13,Kalimantan Selatan,4.8 Mw,10.0,VI,,,Banyak rumah rusak di Kabupaten Banjar
5,2024-01-01,Jawa Barat,4.5 Mw,10.0,IV-V,,11,"Gempa bumi swarm. 1,005 rumah rusak di Sumedan..."
6,2023-12-31,Jawa Barat,4.8 Mw,12.0,V,,3,"Gempa bumi swarm. 1,005 rumah rusak di Sumedan..."
7,2023-11-22,Halmahera,6.0 Mw,102.0,VI,1,,[12][13]
8,2023-06-30,Yogyakarta,5.9 Mw,76.4,IV,1,22,Gempa bumi Yogyakarta 2023
9,2023-04-25,Sumatera Barat,7.1 Mw,34.0,VIII,,,Gempa bumi Kepulauan Mentawai April 2023


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 219 entries, 0 to 218
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Tanggal         219 non-null    object
 1   Lokasi          219 non-null    object
 2   Kekuatan        219 non-null    object
 3   Kedalaman (km)  219 non-null    object
 4   MMI             219 non-null    object
 5   Korban Jiwa     219 non-null    object
 6   Korban Luka     219 non-null    object
 7   Keterangan      219 non-null    object
dtypes: object(8)
memory usage: 15.4+ KB


<h1>PROSES TRANSFORM DATA</h1>

<p>Terdapat data yang ternyata dari sumbernya berbeda jumlah kolomnya. Data tersebut mulai dari index 154 hingga akhir</p>

In [57]:
df.loc[154: ]

,Tanggal,Lokasi,Kekuatan,Kedalaman (km),MMI,Korban Jiwa,Korban Luka,Keterangan
154,1899-09-30,Pulau Seram,7.8 Ms,25.0,VIII,3.864,Gempa bumi Pulau Seram 1899,
155,1898-01-06,Ambon,7.8 Mw,10.0,IX,126,[50],
156,1896-08-15,Jawa Timur,6.7 Mw,90.0,VII,,,
157,1896-07-01,Jawa Timur,6.3 Mw,80.0,VI,,,
158,1896-04-18,Timor-Alor,7.0 Mw,10.0,VIII,250,Tsunami,
...,...,...,...,...,...,...,...,...
214,1644-05-17,Maluku,M6.8,10.0,VII,,Gempa bumi doublet. Beberapa bangunan rusak.,
215,1644-05-12,Maluku,7.0,10.0,VII,,Gempa bumi doublet. Beberapa bangunan rusak.,
216,1631,Mentawai,8.2,20.0,VII,,,
217,1629-08-01,Maluku,8.5 Mw,25.0,VIII-IX,5,Gempa bumi Laut Banda 1629,


Proses penggeseran kolom dan mengganti nilai kolom yang digeser

In [58]:
df.loc[154: , 'Keterangan'] = df.loc[154: , 'Korban Luka'].values
df.loc[154: , 'Korban Luka'] = '0'
df.loc[154: ]

,Tanggal,Lokasi,Kekuatan,Kedalaman (km),MMI,Korban Jiwa,Korban Luka,Keterangan
154,1899-09-30,Pulau Seram,7.8 Ms,25.0,VIII,3.864,0,Gempa bumi Pulau Seram 1899
155,1898-01-06,Ambon,7.8 Mw,10.0,IX,126,0,[50]
156,1896-08-15,Jawa Timur,6.7 Mw,90.0,VII,,0,
157,1896-07-01,Jawa Timur,6.3 Mw,80.0,VI,,0,
158,1896-04-18,Timor-Alor,7.0 Mw,10.0,VIII,250,0,Tsunami
...,...,...,...,...,...,...,...,...
214,1644-05-17,Maluku,M6.8,10.0,VII,,0,Gempa bumi doublet. Beberapa bangunan rusak.
215,1644-05-12,Maluku,7.0,10.0,VII,,0,Gempa bumi doublet. Beberapa bangunan rusak.
216,1631,Mentawai,8.2,20.0,VII,,0,
217,1629-08-01,Maluku,8.5 Mw,25.0,VIII-IX,5,0,Gempa bumi Laut Banda 1629


Cleaning

In [59]:
df['Korban Jiwa'] = df['Korban Jiwa'].str.strip().str.replace('.','', regex=False)
df['Korban Luka'] = df['Korban Luka'].str.strip().str.replace('.','', regex=False)

In [60]:
df.head()

,Tanggal,Lokasi,Kekuatan,Kedalaman (km),MMI,Korban Jiwa,Korban Luka,Keterangan
0,2024-09-18,Jawa Barat,5.1 Mw,10.0,VI,2,159,Gempa bumi Jawa Barat 2024
1,2024-09-07,Bali,4.8 Mw,35.8,VI,1,3,10 bangunan rusak.[6][7]
2,2024-04-27,Jawa Barat,6.1 Mw,54.3,V,1,12,Ratusan bangunan rusak.[8][9]
3,2024-03-22,Jawa Timur,6.4 Mw[10],9.5,VIII,,4,Gempa bumi Laut Jawa 2024
4,2024-02-13,Kalimantan Selatan,4.8 Mw,10.0,VI,,,Banyak rumah rusak di Kabupaten Banjar


In [61]:
df.to_csv('data_gempa_Indonesia.csv', index=False)